## Basins of attraction of a fast-slow multistable model

The goal of this exercise is to demonstrate that `AttractorsViaRecurrences` is not magic, and has its own limitations. 

Consider the following (modified) form of the Rozenweig-MacArthur continuous time model for predator-prey dynamics:
$$

\begin{aligned}
\frac{dN}{dt} & = rN\left(1-\frac{c}{r}N\right)\left(\frac{N-\mu}{N+\nu}\right) - A \\
\frac{dP}{dt} & = \chi A - \delta P \\
A & = \frac{\alpha N P}{\beta + N} 
\end{aligned}
$$

with initial parameters `p = [2.0, 0.19, 0.03, 0.003, 800, 1.5, 0.004, 2.2] = r, c, μ, ν, α, β, χ, δ`. 

After creating the dynamical system instance, create an `AttractorsViaRecurrences` instance, using the following grid. Set your own choice for `consecutive_recurrences`, which is the keyword that decides how many recurrences we should count before declaring convergence to an attractor.

```julia
density = 101
xg = range(-0.1, 20; length = density)
yg = range(-0.0001, 0.03; length = density)
grid = (xg, yg)

mapper = AttractorsViaRecurrences(
    ds, grid;
    consecutive_recurrences = YOUR_CHOICE, Δt = 0.01,
)
```

Then, use the function `basins_of_attraction` to estimate the basins of attraction.
For estimating the basins, avoid initial conditions with ≤ 0 values, i.e., use 
```julia
basinsgrid = (xg[2:end], yg[2:end])
basins, attractors = basins_of_attraction(mapper, basinsgrid)
```
You may visualize these results using 
```julia
fig = heatmap_basins_attractors(basinsgrid, basins, attractors)
```

Repeat this calculation while varying `consecutive_recurrences`. You will notice that for low values of `consecutive_recurrences` the computation fails with an informative error. Can you understand why? Use the `interactive_trajectory_timeseries` GUI app of DynamicalSystems.jl and evolve some initial conditions to help you comprehend the problem.

## Continuation of basins fractions and Hopf bifurcation identification

Continue from the previous exercise. To ensure accurate attractor identification, use the following keywords for `AttractorsViaRecurrences`
```julia
consecutive_recurrences = 5000, Δt = 0.1,
force_non_adaptive = true, Ttr = 100.0,
```
Use the `continuation` function to continue attractors and their fractions across the parameter range `r = 1` to `2`. Use the `statespace_sampler` function to generate random initial conditions in the `basinsgrid` you defined above. Plot the fractions of basins of attraction versus the parameter range, and the attractor value, using the following function to map attractors to real numbers:
```julia
using Statistics: mean
attractor_to_real = A -> mean(x[1] for x in A)
```

The solution of the exercise should be that you have only two attractors and the fractions of their basins are approximately 50% all the time.

One of the two attractors is a fixed point that becomes a limit cycle, i.e., it undergoes a Hopf bifurcation. Find the parameter value where this occurs by analyzing the attractors corresponding to the ID of the "fixed point -> limit cycle" attractor. Identify the bifurcation point by checking when then `length` of the attractor becomes larger than 1. (this means, the attractor spans more than 1 cells in the tesselated state space)

## Basin instability

_Basin instability_ is a simple concept used in dynamical systems, and in particular in the situation of _rate-induced tipping_, see e.g., Ritchie et al., Earth Syst. Dynam., 14, 669–683, https://doi.org/10.5194/esd-14-669-2023, 2023. 

The definition is simple: basin instability is a property of an attractor, and quantifies its behavior between two parameters `p1, p2` of the dynamical system. If the points of the attractor A at parameter `p1` reside in the basin of _another_ attractor B at parameter `p2` then A is basin unstable.

Compute this concept continuing from the Rozenweig-MacArthur predator prey model. Consider the parameters `p1 = r = 2.5` and `p2 = r = 1.8`. The limit cycle attractor of the sytem at these two parameters is basin unstable. Show this, by first finding the attractor at both parameters (using the same `AttractorsViaRecurrences` mapper as before), and then matching the attractors at parameters `p1, p2` to ensure that both limit cycles have the same ID. To compute the basin instability simply count how many points of the limit cycle at `p1` are in the basin of the non-limit cycle attractor at `p2`.

## Alternative ways to match attractors

TODO: Basin enclosure matching fits perfectly in the above exercise!

## Minimal fatal shock for generic attractors

The minimal fatal shock (MFS) is the minimum-magnitude perturbation required for a fixed point attractor to be brought outside its own basin of attraction. It can be estimated with the `minimal_fatal_shock` function. While the concept makes sense for fixed point attractors, for limit cycles or chaotic attractors it is less useful as there is no specific state on the attractor from which it is the most relevant to obtain the MFS. In such cases one would have to obtain MFSs for many points on the attractor. This exercise is about doing exactly that!

Consider the Thomas cyclical system:

$$
\begin{aligned}
\dot{x} &= \sin(y) - bx\\
\dot{y} &= \sin(z) - by\\
\dot{z} &= \sin(x) - bz
\end{aligned}
$$

for $b=0.17$. At this parameter the system has 3 co-existing limit cycle attractors. 
Create a `mapper = AttractorsViaRecurrences(...)` with grid

```julia
xg = yg = zg = range(-6.0, 6.0; length = 100)
grid = (xg, yg, zg)
```

leaving all other metaparameters default. With this `mapper` obtain any of the three attractors of the system by doing `id = mapper(rand(3))`, and then `A = extract_attractors(mapper)[id]`. 

Calculate the MFS for various states on the attractor `A` and collect all these MFSs in a vector. What is the average shock magnitude? Plot a scatter plot of the shock mangitude on the attractor (i.e., scatterplot the points of `A` while assigning as color or markersize the mangitude of the corresponding MFS).

_Hint: For the minimal fatal shock you can keep things simple and use as options:_
```julia
mfs_algo = MFSBruteForce(1000, 1000, 0.99)
search_area = (-6.0, 6.0)
```